In [2]:
import re
import pandas as pd
import numpy as np
import json
from bs4 import BeautifulSoup
from nltk.corpus import stopwords #불용어 제거 모듈
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer

In [3]:
DATA_IN_PATH = './data/'
train_data = pd.read_csv(DATA_IN_PATH + 'labeledTrainData.tsv', header=0, delimiter='\t', quoting=3)
print(train_data['review'][0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<br /><br />Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.<br /><br />The actual feature film bit when it finally sta

In [6]:
def preprocessing(review, remove_stopwords = False):
    # 불용어 제거는 옵션으로 선택 가능
    
    # 1. HTML 태그 제거
    review_text = BeautifulSoup(review, 'html5lib').get_text()
    
    # 2. 영어가 아닌 특수문자를 공백으로 변경
    review_text = re.sub('[^a-zA-Z]', ' ', review_text)
    
    # 3. 대문자를 소문자로 바꾸고 공백 단위로 텍스트를 나눔
    words = review_text.lower().split()
    
    if remove_stopwords:
        # 4. 불용어 제거
        stops = set(stopwords.words('english'))
        # 영어 불용어 불러오기
        words = [w for w in words if not w in stops]
        # 불용어가 아닌 단어로 이루어진 새로운 리스트 생성
        
        # 5. 단어 리스트를 공백을 넣어서 하나의 글로 합침
        clean_review = ' '.join(words)
        
    else: # 불용어를 제거하지 않을 때
        clean_review = ' '.join(words)
    
    return clean_review

In [7]:
clean_train_reviews = []
for review in train_data['review']:
    clean_train_reviews.append(preprocessing(review, remove_stopwords = True))

In [8]:
clean_train_reviews[0]

'stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate workin

## 전처리한 데이터에서 각 단어를 인덱스로 벡터화

In [10]:
clean_train_df = pd.DataFrame({'review':clean_train_reviews, 'sentiment':train_data['sentiment']})
clean_train_df.head()
# 전처리한 데이터를 데이터프레임으로 변환

,review,sentiment
0,stuff going moment mj started listening music ...,1
1,classic war worlds timothy hines entertaining ...,1
2,film starts manager nicholas bell giving welco...,0
3,must assumed praised film greatest filmed oper...,0
4,superbly trashy wondrously unpretentious explo...,1


In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_reviews)
text_sequences = tokenizer.texts_to_sequences(clean_train_reviews)
print(text_sequences[0])
# 모델에 따라 입력값이 텍스트가 아닌 각 단어의 인덱스로 되어 있어야 한다.

[404, 70, 419, 8815, 506, 2456, 115, 54, 873, 516, 178, 18686, 178, 11242, 165, 78, 14, 662, 2457, 117, 92, 10, 499, 4074, 165, 22, 210, 581, 2333, 1194, 11242, 71, 4826, 71, 635, 2, 253, 70, 11, 302, 1663, 486, 1144, 3265, 8815, 411, 793, 3342, 17, 441, 600, 1500, 15, 4424, 1851, 998, 146, 342, 1442, 743, 2424, 4, 8815, 418, 70, 637, 69, 237, 94, 541, 8815, 26055, 26056, 120, 1, 8815, 323, 8, 47, 20, 323, 167, 10, 207, 633, 635, 2, 116, 291, 382, 121, 15535, 3315, 1501, 574, 734, 10013, 923, 11578, 822, 1239, 1408, 360, 8815, 221, 15, 576, 8815, 22224, 2274, 13426, 734, 10013, 27, 28606, 340, 16, 41, 18687, 1500, 388, 11243, 165, 3962, 8815, 115, 627, 499, 79, 4, 8815, 1430, 380, 2163, 114, 1919, 2503, 574, 17, 60, 100, 4875, 5100, 260, 1268, 26057, 15, 574, 493, 744, 637, 631, 3, 394, 164, 446, 114, 615, 3266, 1160, 684, 48, 1175, 224, 1, 16, 4, 8815, 3, 507, 62, 25, 16, 640, 133, 231, 95, 7426, 600, 3439, 8815, 37248, 1864, 1, 128, 342, 1442, 247, 3, 865, 16, 42, 1487, 997, 2333, 12

In [12]:
word_vocab = tokenizer.word_index
#print(word_vocab)
# 각 인덱스가 어떤 단어를 의미하는지 확인이 가능해야 하기 때문에 단어 사전이 필요

In [13]:
print("전체 단어 개수: ", len(word_vocab)+1)

전체 단어 개수:  74066


In [14]:
data_configs = {}

data_configs['vocab'] = word_vocab # 단어 사전
data_configs['vocab_size'] = len(word_vocab) # 전체 단어의 개수
# 모델에서 사용하기 위해 따로 저장

## 데이터 길이를 동일하게 해주는 패딩 과정

In [15]:
MAX_SEQUENCE_LENGTH = 174 # 문장 최대 길이, 174인 이유는 EDA 과정에서 나온 중간값

train_inputs = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')
# pad_sequences(데이터, 문장 최대 길이, 패딩 = 'post' : 패딩할 0값을 뒤에 붙임 )
print('Shape of train data :', train_inputs.shape)

Shape of train data : (25000, 174)


## 정답 데이터를 넘파이 배열로 변환

In [16]:
train_labels = np.array(train_data['sentiment'])
print('Shape of label tensor:', train_labels.shape)

Shape of label tensor: (25000,)


In [17]:
# 넘파이 배열로 변환하는 이유는 전처리한 데이터를 저장할 때 넘파이 형태로 저장하기 때문

## 데이터의 저장

In [17]:
TRAIN_INPUT_DATA = 'train_input.npy'
TRAIN_LABEL_DATA = 'train_label.npy'
TRAIN_CLEAN_DATA = 'train_clean.csv'
DATA_CONFIGS = 'data_configs.json'

import os
# 저장하는 디렉토리가 존재하지 않으면 생성
if not os.path.exists(DATA_IN_PATH):
    os.makedirs(DATA_IN_PATH)

In [18]:
# 전처리 된 데이터를 넘파이 형태로 저장
np.save(open(DATA_IN_PATH + TRAIN_INPUT_DATA, 'wb'), train_inputs)
np.save(open(DATA_IN_PATH + TRAIN_LABEL_DATA, 'wb'), train_labels)

# 정제된 텍스트를 csv 형태로 저장
clean_train_df.to_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA, index = False)


# 데이터 사전을 json 형태로 저장
json.dump(data_configs, open(DATA_IN_PATH + DATA_CONFIGS, 'w'), ensure_ascii=False)

## 평가 데이터의 전처리 및 저장

In [19]:
test_data = pd.read_csv(DATA_IN_PATH + "testData.tsv", header=0, delimiter="\t", quoting=3)

clean_test_reviews = []
for review in test_data['review']:
    clean_test_reviews.append(preprocessing(review, remove_stopwords = True))


clean_test_df = pd.DataFrame({'review': clean_test_reviews, 'id': test_data['id']})
test_id = np.array(test_data['id'])

text_sequences = tokenizer.texts_to_sequences(clean_test_reviews)
# 평가 데이터를 전처리 할 때, 학습 데이터에 적용한 토크나이저 객체를 사용해야 한다.
# fit_on_texts의 과정이 없음
# 새롭게 만들면 학습 데이터와 평가 데이터에 대한 각 단어들의 인덱스가 달라져서 모델에 정상적으로 적용 불가.
test_inputs = pad_sequences(text_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post')

In [20]:
TEST_INPUT_DATA = 'test_input.npy'
TEST_CLEAN_DATA = 'test_clean.csv'
TEST_ID_DATA = 'test_id.npy'

np.save(open(DATA_IN_PATH + TEST_INPUT_DATA, 'wb'), test_inputs)
np.save(open(DATA_IN_PATH + TEST_ID_DATA, 'wb'), test_id)
clean_test_df.to_csv(DATA_IN_PATH + TEST_CLEAN_DATA, index = False)